## 1. Defining the Question

 analysis of some stock market data on Safaricom from the years 2012-2017






In [2]:
# Installing pyspark
# ---
#
!pip install pyspark

# Next, we run a local spark session
# ---
#
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2e6c80fb64f564b5d5dacfe7fea179b2199974dd06c4c93b4381f922b2a395db
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
spark

In [4]:
# Import a Spark function from library
from pyspark.sql.functions import col

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

**Data Importation and Exploration**


In [6]:
from google.colab import files
 
 
uploaded = files.upload()

Saving saf_stock.csv to saf_stock.csv


**Show the first 5 rows**

In [7]:
df = spark.read.csv("saf_stock.csv", header=True, inferSchema=True)
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



 Determine the column names



In [8]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



observation 

we have 2 data types 


1.  DoubleType 
2.   IntegerType 




In [9]:
df.count()

1258

In [10]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

**Data Preparation**


Format all the data to 2 decimal places

In [11]:
from pyspark.sql.functions import round, col
import pyspark.sql.functions as F
for c_name, c_type in df.dtypes:
    if c_type in ('double', 'float'):
        df = df.withColumn(c_name, F.round(c_name, 2))
df.show()

+-------------------+-----+-----+-----+-----+--------+---------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|
+-------------------+-----+-----+-----+-----+--------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|
|2012-01-06 00:00:00|59.42|59.45|58.87| 59.0| 8069400|    51.46|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|
|2012-01-10 00:00:00|59.43|59.71|58.98|59.04| 6907300|    51.49|
|2012-01-11 00:00:00|59.06|59.53|59.04| 59.4| 6365600|    51.81|
|2012-01-12 00:00:00|59.79| 60.0| 59.4| 59.5| 7236400|     51.9|
|2012-01-13 00:00:00|59.18|59.61|59.01|59.54| 7729300|    51.93|
|2012-01-17 00:00:00|59.87|60.11|59.52|59.85| 8500000|     52.2|
|2012-01-18 00:00:00|59.79|60.03|59.65|60.01| 5911400|    52.34|
|2012-01-19 00:00:00|59.93|60.73|59.75|60.61| 9234600|    52.86|
|2012-01-20 00:00:00|60.7

Create a new data frame meso a column called HV Ratio that is the ratio of the
High Price versus volume of stock traded for a day

In [12]:
from pyspark.sql.functions import lit
df.withColumn("HV Ratio", df['High']/ df['Volume']) \
  .show()


+-------------------+-----+-----+-----+-----+--------+---------+--------------------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|            HV Ratio|
+-------------------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|4.819714574387472E-6|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|6.290848821573389...|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|4.669413073103491E-6|
|2012-01-06 00:00:00|59.42|59.45|58.87| 59.0| 8069400|    51.46|7.367338339901356E-6|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|8.915604928660188E-6|
|2012-01-10 00:00:00|59.43|59.71|58.98|59.04| 6907300|    51.49|8.644477581688938E-6|
|2012-01-11 00:00:00|59.06|59.53|59.04| 59.4| 6365600|    51.81| 9.35182857861003E-6|
|2012-01-12 00:00:00|59.79| 60.0| 59.4| 59.5| 7236400|     51.9| 8.29141562102703E-6|
|2012-01-13 00:00:00|59.18|59.61|59.01|59.54| 7729300|

Data analysis

In [13]:
df[['date','High']].sort(df.High.desc()).show(truncate=False)

+-------------------+-----+
|date               |High |
+-------------------+-----+
|2015-01-13 00:00:00|90.97|
|2015-01-08 00:00:00|90.67|
|2015-01-09 00:00:00|90.39|
|2015-01-12 00:00:00|90.31|
|2015-01-23 00:00:00|89.26|
|2015-01-26 00:00:00|89.16|
|2015-01-07 00:00:00|88.68|
|2015-01-14 00:00:00|88.52|
|2015-01-27 00:00:00|88.46|
|2015-01-22 00:00:00|88.4 |
|2015-01-28 00:00:00|88.23|
|2014-11-28 00:00:00|88.09|
|2015-02-06 00:00:00|88.0 |
|2015-01-15 00:00:00|87.78|
|2015-01-29 00:00:00|87.72|
|2015-01-20 00:00:00|87.7 |
|2015-01-16 00:00:00|87.46|
|2014-12-31 00:00:00|87.44|
|2015-02-10 00:00:00|87.41|
|2015-01-30 00:00:00|87.36|
+-------------------+-----+
only showing top 20 rows



2015-01-13  had  Peak High in Price of 90.97

In [14]:
df[['close']].describe().show()

+-------+-----------------+
|summary|            close|
+-------+-----------------+
|  count|             1258|
|   mean|72.38844992050863|
| stddev|6.756859155425468|
|    min|            56.42|
|    max|            90.47|
+-------+-----------------+



Mean of the close columns is 72.388

In [15]:
df[['Volume']].describe().show()

+-------+-----------------+
|summary|           Volume|
+-------+-----------------+
|  count|             1258|
|   mean|8222093.481717011|
| stddev|  4519780.8431556|
|    min|          2094900|
|    max|         80898100|
+-------+-----------------+



min 2094900 and max is 80898100

In [16]:
less_meso = df[df['close'] < 60]
less_meso.count()

81

How many days was the Close lower than 60 dollars 81 days 

In [17]:
per = df[df['High'] > 80]
per.count()
percentage = (100*per.count()/df.count())
percentage

9.141494435612083

What percentage of the time was the High greater than 80 dollars? 9.14%

In [18]:
df[['High']].describe().show()

+-------+-----------------+
|summary|             High|
+-------+-----------------+
|  count|             1258|
|   mean|72.83938791732918|
| stddev|6.768186744823553|
|    min|            57.06|
|    max|            90.97|
+-------+-----------------+

